In [1]:
import logging
import os
import openai
import itertools
import inspect


from actionweaver.llms.azure.chat import ChatCompletion
from actionweaver.llms.azure.tokens import TokenUsageTracker
from actionweaver import action



In [2]:
logging.basicConfig(
    filename='agent.log',
    filemode='a',
    format='%(asctime)s.%(msecs)04d %(levelname)s {%(module)s} [%(funcName)s] %(message)s',
    level=logging.INFO,
    datefmt='%Y-%m-%d %H:%M:%S'
)

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

In [3]:
@action(name="GetCurrentTime")
def get_current_time() -> str:
    """
    Use this for getting the current time in the specified time zone.
    
    :return: A string representing the current time in the specified time zone.
    """
    import datetime
    current_time = datetime.datetime.now()
    
    return f"The current time is {current_time}"

In [4]:
chat = ChatCompletion(
    model="gpt-35-turbo-0613-16k", 
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
    api_key=os.getenv("AZURE_OPENAI_KEY"),  
    api_version="2023-10-01-preview", 
    token_usage_tracker = TokenUsageTracker(budget=2000, logger=logger), 
    logger=logger)

In [5]:
def print_output(output):
    from collections.abc import Iterable
    if isinstance(output, str):
        print (output)
    elif isinstance(output, Iterable):
        for chunk in output:
            content = chunk.choices[0].delta.content
            if content is not None:
                print(content, end='')

In [21]:
print_output(chat.create([{"role": "user", "content": "what time is it"}], actions = [], stream=False))

I'm sorry, I am an AI language model and I do not have access to real-time information. Please check the time on your device or ask someone nearby for the current time.


In [22]:
print_output(chat.create([{"role": "user", "content": "what time is it"}], actions = [get_current_time], stream=False))

The current time is 19:10 (7:10 PM).


In [23]:
print_output(chat.create([{"role": "user", "content": "what time is it"}], actions = [], stream=True))

I'm sorry, I am an AI language model and I do not have access to real-time information. Please check the time on your device or ask someone nearby for the current time.

In [20]:
print_output(chat.create([{"role": "user", "content": "what time is it, print in chinese"}], actions = [get_current_time], stream=True))

现在的时间是2023年11月11日19:09:56.316740。